In [1]:
μ_min1 = Array{Float64}(undef, 2, 2, 2, 2)
π0 = Array{Float64}(undef, 2 ,2 , 2, 2)
β = 0.8
ν = 1.1

1.1

In [2]:
for i in 1:2
    for j in 1:2
        matrix = rand(2, 2)
        matrix2 = matrix/sum(matrix)
        μ_min1[i, j, :, :] = matrix2
    end
end
μ_min1

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 0.361376   0.0627167
 0.0262126  0.0396221

[:, :, 2, 1] =
 0.281423  0.130466
 0.374074  0.465261

[:, :, 1, 2] =
 0.0767369  0.492714
 0.331151   0.0355924

[:, :, 2, 2] =
 0.280464  0.314104
 0.268563  0.459524

In [3]:
for i in 1:2
    for j in 1:2
        matrix = rand(2, 2)
        matrix2 = matrix/sum(matrix)
        π0[i, j, :, :] = matrix2
    end
end
π0

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 0.131254  0.481893
 0.172591  0.00845185

[:, :, 2, 1] =
 0.453227  0.193638
 0.409105  0.43712

[:, :, 1, 2] =
 0.10609  0.166756
 0.17543  0.471137

[:, :, 2, 2] =
 0.30943   0.157714
 0.242873  0.0832904

In [4]:
κdot = rand(0.95:0.001:1.08, (2, 2, 2, 2))

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 0.982  1.04
 0.996  1.008

[:, :, 2, 1] =
 0.979  1.067
 1.027  1.006

[:, :, 1, 2] =
 1.077  0.99
 1.002  1.019

[:, :, 2, 2] =
 0.957  1.019
 1.038  0.97

In [5]:
Ȧ = rand(0.98:0.001:1.05, (2, 2))

2×2 Matrix{Float64}:
 1.002  1.026
 1.044  0.992

In [6]:
L_setup_matrix = rand(2, 2)
L0 = L_setup_matrix/sum(L_setup_matrix)

2×2 Matrix{Float64}:
 0.10845   0.199478
 0.375173  0.316899

In [7]:
w0 = rand(0.5:0.001:1.5, (2, 2))

2×2 Matrix{Float64}:
 1.203  1.333
 0.682  0.903

In [8]:
using Distributions
θ_var = Frechet()
θ_list = rand(θ_var, 2)

2-element Vector{Float64}:
 0.19239663457364212
 2.576172013142694

In [ ]:
u̇_0t = Array{Float64}(undef, 2, 2, 2)
u̇_0t[:, :, 1] = rand(0.98:0.001:1.05, (2, 2))
u̇_0t[:, :, 2] = ones(2, 2)
println(u̇_0t)
ẇ = rand(0.97:0.001:1.06, (2, 2))
μ_0 = Array{Float64}(undef, 2, 2, 2, 2)
L1 = Array{Float64}(undef, 2, 2)
ẇn = Array{Float64}(undef, 2, 2)
Ṗ = Array{Float64}(undef, 2, 2)
π1 = Array{Float64}(undef, 2, 2, 2, 2)
X = Array{Float64}(undef, 2, 2)
α = Array{Float64}(undef, 2)
Xt1 = Array{Float64}(undef, 2, 2)

while !all(0.99 .* u̇_0t[:, :, 2] .<= u̇_0t[:, :, 1].<= 1.01 .* u̇_0t[:, :, 2])
#    μ_0 = Array{Float64}(undef, 2, 2, 2, 2)
    for n in 1:2
        for j in 1:2
            for i in 1:2
                for k in 1:2
                    denom = sum(μ_min1[n, j, p, q]*((u̇_0t[p, q, 2])^(β/ν)) for p in 1:2, q in 1:2)
                    μ_0[n, j, i, k] = (μ_min1[n, j, i, k]*((u̇_0t[i, k, 2])^(β/ν)))/denom
                end
            end
        end
    end
#    L1 = Array{Float64}(undef, 2, 2)
    for n in 1:2
        for j in 1:2
            L1[n, j] = sum(μ_0[i, k, n, j]*L0[i, k] for i in 1:2, k in 1:2)
        end
    end
#    ẇn = Array{Float64}(undef, 2, 2)
    while !all(0.99*ẇ .<= ẇn .<= 1.01*ẇ)

#        Ṗ = Array{Float64}(undef, 2, 2)
        for n in 1:2
            for i in 1:2
                for j in 1:2
                    Ṗ[n, j] = (sum(π0[n, j, i, j]*((ẇ[i, j]*κdot[n, j, i, j])^(-θ_list[j]))*((Ȧ[i, j])^θ_list[j]) for i in 1:2))^(-1/θ_list[j])
                end
            end
        end
#        π1 = Array{Float64}(undef, 2, 2, 2, 2)
        for n in 1:2
            for i in 1:2
                for j in 1:2
                    π1[n, j, i, j] = π0[n, j, i, j]*(((ẇ[i, j]*κdot[n, j, i, j])/Ṗ[n, j])^(-θ_list[j]))*((Ȧ[i,j])^θ_list[j])
                end
            end
        end
        for i in 1:2
            for j in 1:2
                X[i, j] = w0[i, j]*L0[i, j]
            end
        end
        for i in 1:2
            α[i] = sum(X[k, i] for k in 1:2)/sum(X[j, l] for j in 1:2, l in 1:2)
        end
#        Xt1 = Array{Float64}(undef, 2, 2)
        for n in 1:2
            for j in 1:2
                Xt1[n, j] = α[j]*sum(ẇ[n, k]*L1[n, k]*w0[n, k] for k in 1:2)
            end
        end
        for n in 1:2
            for j in 1:2
                ẇn[n, j] = sum(π1[i, j, n, j]*Xt1[i, j] for i in 1:2)/(L1[n, j]*w0[n, j])
            end
        end
        ẇ = ẇn
    end
    u̇_1t = Array{Float64}(undef, 2, 2)
    for n in 1:2
        for j in 1:2
            u̇_1t[n, j] = ẇ[n, j]*(sum(μ_0[n, j, i, k]*(u̇_0t[i, k, 2])^(β/ν) for i in 1:2, k in 1:2))^(ν)
        end
    end
    
    global u̇_0t[:, :, 1] = u̇_1t
end 

[

In [ ]:
u̇_0t